Bibliografia:

Para o modelo de submissão:
https://www.kaggle.com/code/philculliton/nlp-getting-started-tutorial/notebook

Conceitos de ML:
https://tatianaesc.medium.com/machine-learning-conceitos-e-modelos-f0373bf4f445

Visualização:
https://realpython.com/twitter-sentiment-python-docker-elasticsearch-kibana/

Conceito VADER:
https://ojs.aaai.org/index.php/ICWSM/article/view/14550/14399

algoritmo de classificação utilizado no titanic
wording bading / baggle of words/ df-tf (atribui pesos as palvras)
feater extraction -> transformar o texto em uma coluna de banco de dados

1)Limpeza das palavras: tirar as stop words, lematização, Bag-of-words

## 1) Vizualização dos dados

In [4]:
import numpy as np
import pandas as pd
import nltk
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import sent_tokenize, word_tokenize

In [56]:
from sklearn.feature_extraction.text import CountVectorizer

In [21]:
lemmatizer = WordNetLemmatizer()

In [5]:
from nltk.stem import WordNetLemmatizer

In [6]:
from nltk.corpus import stopwords

In [55]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [46]:
train_clean

,id,keyword,location,text,target,text clean
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,"['deed', 'reason', 'earthquake', 'may', 'allah..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,"['forest', 'fire', 'near', 'la', 'ronge', 'sas..."
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,"['resident', 'asked', ""'shelter"", 'place', ""'""..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"['people', 'receive', 'wildfire', 'evacuation'..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,"['got', 'sent', 'photo', 'ruby', 'alaska', 'sm..."
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,"['two', 'giant', 'crane', 'holding', 'bridge',..."
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,"['control', 'wild', 'fire', 'california', 'eve..."
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,"['utckm', 'volcano', 'hawaii']"
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,"['police', 'investigating', 'e-bike', 'collide..."


In [8]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [9]:
tweets = train_df['text']

## 2) Minimização dos ruídos

### Substituindo os espaços e lemmatizando as palavras chaves

In [10]:
filtro = train_df['tweets_lemmatizedword'].notnull()

KeyError: 'tweets_lemmatizedword'

In [ ]:
sample_df_nonull=train_df[filtro]

In [ ]:
len(sample_df_nonull)

7552

In [ ]:
tweets_lemmatizedword = sample_df_nonull['tweets_lemmatizedword']

In [ ]:
tweets_lemmatizedwords = [x.replace('%20', ' ') for x in tweets_lemmatizedword]

In [20]:
lemmatizer = WordNetLemmatizer()

In [ ]:
tweets_lemmatizedwords_words = [word_tokenize(x) for x in tweets_lemmatizedwords]

In [ ]:
tweets_lemmatized= []
for tweets_lemmatizedwords in tweets_lemmatizedwords_words:
    words_lemmatized = []
    for word in tweets_lemmatizedwords:
        words_lemmatized.append(lemmatizer.lemmatize(word))
    tweets_lemmatized.append(words_lemmatized)    

In [ ]:
sample_df_nonull['tweets_lemmatizedword'] = tweets_lemmatized

C:\Users\yfrom\AppData\Local\Temp\ipykernel_11348\2651402788.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_df_nonull['keyword'] = key


### Text

In [11]:
tweets=train_df['text']

In [12]:
#Retirada dos links
tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')

C:\Users\yfrom\AppData\Local\Temp\ipykernel_15608\100282221.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+/(?:[-\w.]|(?:%[\da-fA-F]{2}))+','')


In [13]:
#Retirada dos @
tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')

C:\Users\yfrom\AppData\Local\Temp\ipykernel_15608\2578260738.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'@[a-zA-Z0-9_!.]{0,25}','')


In [66]:
#retirada de símbolos não alfanuméticos 
tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")

C:\Users\yfrom\AppData\Local\Temp\ipykernel_15608\192283893.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r"[#,.;:_?!()\[\]]","")


In [15]:
#retirada de \\x89Û_ \x89ÛÒ \x89Û \x89Û÷....(Não sei o que significa..e tem vários desse tipo)
tweets = tweets.replace("\\x89Û_",'')
tweets = tweets.replace('\x89ÛÓ','')
tweets = tweets.replace('\x89ÛÓ','')
tweets = tweets.replace('\x89ÛÒ','')
tweets = tweets.replace('\x89Û','')
tweets = tweets.replace('\x89Û÷','')
tweets = tweets.replace('\x89ûï','')

In [16]:
tweets = tweets.str.replace(r'[0-9]','')

C:\Users\yfrom\AppData\Local\Temp\ipykernel_15608\2767507254.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  tweets = tweets.str.replace(r'[0-9]','')


In [17]:
tweets = tweets.str.lower()

In [18]:
stop_words = set(stopwords.words("english"))

In [22]:
sample_tweets= []
for tweet in tweets:
    filtered_list = []
    words_lemmatized = []
    words = []
    words = word_tokenize(tweet) 
    words_lemmatized = [lemmatizer.lemmatize(word) for word in words] 
    for word in words_lemmatized:
        #print(word)
        if word not in stop_words:
            filtered_list.append(word)
    sample_tweets.append(filtered_list)

In [67]:
train_df['text']= tweets
train_df['words']=sample_tweets

In [68]:
train_df.head()

,id,keyword,location,text,target,words
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,"[deed, reason, earthquake, may, allah, forgive..."
1,4,NaN,NaN,forest fire near la ronge sask canada,1,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,all residents asked to 'shelter in place' are ...,1,"[resident, asked, 'shelter, place, ', notified..."
3,6,NaN,NaN,people receive wildfires evacuation orders in...,1,"[people, receive, wildfire, evacuation, order,..."
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,"[got, sent, photo, ruby, alaska, smoke, wildfi..."


## Bag-of-words
 Definição: Bag-of-words model is a way of representing text data when modeling text with machine learning algorithms. Machine learning algorithms cannot work with raw text directly; the text must be converted into well defined fixed-length(vector) numbers.

In [85]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [33]:
dicionario = set()

In [52]:
for tweet in train_df['text']:
    for word in tweet:
        dicionario.add(word)

Formação da Bag-of-word

In [78]:
Vectorizer = CountVectorizer()
Vectorizer.fit(train_df['text'])
vector = Vectorizer.transform(train_df['text'])#matriz esparsa

In [ ]:
# Aplicando configurações no modelo
clf = LogisticRegression(C=30.0, class_weight='balanced', solver='newton-cg',multi_class='multinomial', n_jobs=-1, random_state=40)
#Treinando o modelo com o vocabulário construindo e as variáveis alvo
clf.fit(X_train_counts, y_train)
#Realizando as predições para o conjunto de teste
y_predicted_counts = clf.predict(X_test_counts)